In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
import pandas as pd
import argparse                 
import sklearn 
import numpy as np 
import logging 
import sklearn.metrics as metrics 
import scikitplot as skplt
import matplotlib.pyplot as plt                                  
from argparse import ArgumentParser
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.utils.data import DataLoader, TensorDataset, Subset, Dataset, random_split
import torch.nn.functional as F
from collections import Counter, defaultdict
import itertools
import os 
import copy
import warnings
from dataload import makedata
from classifier import ClassifierTrainer
from models import VQ_Classifier
import itertools
from data import load_data
warnings.filterwarnings("ignore", category=UserWarning)
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
torch.set_num_threads(32) 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(911)

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] ='3'
len_position=13
dataset = 'peak'
classifier ='cnn'
classification_model =  '/home/smjo/xai_timeseries/vqvae/saved_models/classification/peak/2/cnn.pt'
vqvae_model = '/home/smjo/xai_timeseries/vqvae/saved_models/peak/2/model_280.pt'
batch_size=256
ds = load_data(dataset, task='classification')


task is classification
dataset is peak
X shape:torch.Size([800, 192]), y shape:torch.Size([800, 2])


In [11]:
ds[0][0].shape

torch.Size([192])

In [4]:
train_size = int(0.8 * len(ds))
val_size = int(0.1 * len(ds))
test_size = len(ds) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(ds, [train_size, val_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False, pin_memory=True)

In [5]:
#Find masking token
end_tokens={}

conv_net = VQ_Classifier(
    num_classes = 2,
    vqvae_model = vqvae_model,
    positions =0,
    mask = 0,
    auc_classification = False,
    model_type = classifier
).to(device)

a = torch.load(classification_model)
conv_net.load_state_dict(a['model_state_dict'])

for param in conv_net.parameters():
    param.requires_grad = False

conv_net.eval()

VQ_Classifier(
  (vae): Residual_VQVAE(
    (vq): ResidualVQ(
      (layers): ModuleList(
        (0): VectorQuantize(
          (project_in): Identity()
          (project_out): Identity()
          (_codebook): EuclideanCodebook()
        )
        (1): VectorQuantize(
          (project_in): Identity()
          (project_out): Identity()
          (_codebook): EuclideanCodebook()
        )
      )
    )
    (encoder): Sequential(
      (0): Sequential(
        (0): Conv1d(1, 64, kernel_size=(2,), stride=(2,))
        (1): ReLU()
      )
      (1): Sequential(
        (0): Conv1d(64, 64, kernel_size=(2,), stride=(2,))
        (1): ReLU()
      )
      (2): Sequential(
        (0): Conv1d(64, 64, kernel_size=(2,), stride=(2,))
        (1): ReLU()
      )
      (3): Sequential(
        (0): Conv1d(64, 64, kernel_size=(2,), stride=(2,))
        (1): ReLU()
      )
      (4): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
    )
    (decoder): Sequential(
      (0): Sequential(
        (0)

In [6]:
with torch.no_grad():
    for idx, (data, labels) in enumerate(test_loader):
        data = data.unsqueeze(1).float()
        labels = labels.type(torch.LongTensor)
        data, labels = data.to(device), labels.to(device)
        output, codebook_tokens, recon, input= conv_net(data)
        for idx,token in enumerate(codebook_tokens[0]):
            try:
                end_tokens[token[0].item()]+=1
            except:
                end_tokens[token[0].item()]=1
                
    mask_code = max(end_tokens, key=end_tokens.get)
    print(f"mask code is {mask_code} ")
    

80
else case:tensor([[98, 74],
        [98, 74],
        [98, 74],
        [98, 74],
        [98, 74],
        [98, 74],
        [98, 74],
        [ 0, 63],
        [94, 10],
        [18, 75],
        [15,  4],
        [98, 74]], device='cuda:0')
quantized:tensor([[[ 0.4525, -1.0446,  0.3739,  ..., -1.6244,  1.4774,  0.7758],
         [ 0.7811, -1.1668, -0.8728,  ...,  0.8547,  0.4208, -0.7446],
         [ 0.4525, -1.0446,  0.3739,  ..., -1.6244,  1.4774,  0.7758],
         ...,
         [ 1.5036,  0.4498, -0.9438,  ..., -0.5572,  0.8877,  1.2325],
         [ 0.4525, -1.0446,  0.3739,  ..., -1.6244,  1.4774,  0.7758],
         [ 0.7811, -1.1668, -0.8728,  ...,  0.8547,  0.4208, -0.7446]],

        [[ 0.4525, -1.0446,  0.3739,  ..., -1.6244,  1.4774,  0.7758],
         [-0.5975,  1.6595, -0.8026,  ..., -1.1156, -0.6444, -0.8530],
         [ 0.0686, -0.7747,  0.0299,  ..., -0.6927,  1.5212, -1.7734],
         ...,
         [ 0.7811, -1.1668, -0.8728,  ...,  0.8547,  0.4208, -0.7446],
   

In [ ]:
#Make position answer
answer = {}         
arr = [i for i in range(len_position)]
roc_auc_scores=[]


for cnt in range(len_position):
    max_auc=-1
    print("arr:", arr)
    for i in arr:
        #ECG Dataset
        positions = list(answer.values())+[i]
        net = VQ_Classifier(
            num_classes = 2,
            vqvae_model = vqvae_model,
            positions = positions,
            mask = mask_code,
            auc_classification = True,
            model_type = classifier
        ).to(device)
        
        
        net.load_state_dict(torch.load(classification_model)["model_state_dict"])
        net = net.to(device)
        for param in net.parameters():
            param.requires_grad = False        
        net.eval()   

        with torch.no_grad():    
            score=0
            for idx, (data, labels) in enumerate(test_loader):
                data = data.unsqueeze(1).float()
                labels = labels.type(torch.LongTensor)
                labels = torch.argmax(labels, dim=1)
                data, labels = data.to(device), labels.to(device)
                output, codebook_tokens, recon, input= net(data)
            
                score += sklearn.metrics.roc_auc_score(labels.cpu(), output[:,1].cpu())
                
        score = score / (idx+1)
        if score > max_auc:
            answer[f"top_{cnt}"]=i
            max_auc = copy.deepcopy(score)
    roc_auc_scores.append(max_auc)        
    arr.remove(answer[f"top_{cnt}"])

In [16]:
roc_auc_scores

[0.5, 0.5, 0.5, 0.5, 0.5, 0.5125, 0.7, 0.925, 1.0, 1.0, 1.0, 1.0, 1.0]

In [17]:
answer

{'top_0': 0,
 'top_1': 1,
 'top_2': 2,
 'top_3': 3,
 'top_4': 4,
 'top_5': 8,
 'top_6': 6,
 'top_7': 7,
 'top_8': 9,
 'top_9': 5,
 'top_10': 10,
 'top_11': 11,
 'top_12': 12}